In [76]:
import pandas as pd
import numpy as np
from IPython.display import display
import re
from rapidfuzz import fuzz, process
import matplotlib.pyplot as plt
from Own_package import Data_CL_General_py
import importlib
importlib.reload(Data_CL_General_py)

<module 'Own_package.Data_CL_General_py' from 'E:\\Quantime Project\\Own_package\\Data_CL_General_py.py'>

In [5]:
df_purchase_behaviour = pd.read_csv(r"E:\Quantime Project\QVI_purchase_behaviour.csv") # reading the CSV file
df_transaction_data = pd.read_excel(r"E:\Quantime Project\QVI_transaction_data.xlsx") # reading the Excel file

In [6]:
df_purchase_behaviour = Data_CL_General_py.gen_cleaning(df_purchase_behaviour) 
df_transaction_data = Data_CL_General_py.gen_cleaning(df_transaction_data)
# cleaning the 2 dataset with the general_cleaning module

In [7]:
df_purchase_behaviour.head() # just to check if it goes right

,Lylty_Card_Nbr,Lifestage,Premium_Customer
0,1000,Young Singles/Couples,Premium
1,1002,Young Singles/Couples,Mainstream
2,1003,Young Families,Budget
3,1004,Older Singles/Couples,Mainstream
4,1005,Midage Singles/Couples,Mainstream


In [8]:
df_purchase_behaviour[['Age_Stage' , 'Marital' ]] = df_purchase_behaviour['Lifestage'].str.split(' ', n=1 , expand = True) # to split the Lifestage column into 2 columns 
df_purchase_behaviour[['Marital' , 'Fam' ]] = df_purchase_behaviour['Marital'].str.split('/' , n = 1 ,expand= True) # to split the Marital column into 2 columns 
for i in df_purchase_behaviour[['Marital' , 'Fam']]: 
    df_purchase_behaviour[i] = df_purchase_behaviour[i].where(df_purchase_behaviour[i].notnull() , 'Unknown')
# to replace all the NaN values with Unknown

In [9]:
df_purchase_behaviour.drop(columns = 'Lifestage' , inplace= True ) # tp remove the Lifestage column as we have seperated it

In [10]:
df_purchase_behaviour.head() 

,Lylty_Card_Nbr,Premium_Customer,Age_Stage,Marital,Fam
0,1000,Premium,Young,Singles,Couples
1,1002,Mainstream,Young,Singles,Couples
2,1003,Budget,Young,Families,Unknown
3,1004,Mainstream,Older,Singles,Couples
4,1005,Mainstream,Midage,Singles,Couples


In [11]:
df_transaction_data['Prod_Size'] = df_transaction_data.Prod_Name.str[-4:] # New Column with Prod_Size

In [12]:
df_transaction_data['Package_Size']  = df_transaction_data['Prod_Size'] # new columns with package size

In [13]:
df_transaction_data['Prod_Name'] = df_transaction_data.Prod_Name.str.strip().str[:-4] # removing the size from the main column 

In [14]:
df_transaction_data.drop(columns = 'Prod_Size' , inplace=True) # droping the unneeded columns

In [15]:
df_transaction_data.Package_Size.value_counts() # checking if we have anything else Num+G 
# found that we have ""SALT"" & we can't convert it to INT 

Package_Size
175G    66390
150G    43131
134G    25102
110G    22387
170G    19983
165G    15297
300G    15166
330G    12540
380G     6418
270G     6285
210G     6272
200G     4473
Salt     3257
250G     3169
 90G     3008
190G     2995
160G     2970
220G     1564
 70G     1507
180G     1468
125G     1454
Name: count, dtype: int64

In [16]:
df_transaction_data['Package_Size'] = df_transaction_data.Package_Size.str.replace("G",'') # removing the G

In [17]:
round((df_transaction_data[df_transaction_data['Package_Size'] == 'Salt'].shape[0]/ df_transaction_data.value_counts().sum()) * 100,2)
# precentage of 1.2% is not a big deal to be removed but we will let them at the first

1.23

In [18]:
df_transaction_data['Package_Size'] = pd.to_numeric(df_transaction_data['Package_Size'] , errors='coerce') 
# Changing the column to int & the string to NAN

In [19]:
print(f"Mean = {round(df_transaction_data['Package_Size'].mean(),2)}" )# calculating the mean 
print(f"Standard Deviation = {round(df_transaction_data['Package_Size'].std(),2)}") # standard deviation 
print(f"Skew = {round(df_transaction_data['Package_Size'].skew(),2)}") # skew > 1 
# we can't replace the NAN values with mean as already there's skew to the right 

Mean = 183.02
Standard Deviation = 64.51
Skew = 1.43


In [20]:
df_transaction_data.head(2)

,Date,Store_Nbr,Lylty_Card_Nbr,Txn_Id,Prod_Nbr,Prod_Name,Prod_Qty,Tot_Sales,Package_Size
0,2018-10-17,1,1000,1,5,Natural Chip Compny Seasalt,2,6.0,175.0
1,2019-05-14,1,1307,348,66,Ccs Nacho Cheese,3,6.3,175.0


In [21]:
df_transaction_data.dtypes # checking the data-types

Date              datetime64[ns]
Store_Nbr                  int64
Lylty_Card_Nbr             int64
Txn_Id                     int64
Prod_Nbr                   int64
Prod_Name                 object
Prod_Qty                   int64
Tot_Sales                float64
Package_Size             float64
dtype: object

In [22]:
df_transaction_data[['Store_Nbr','Lylty_Card_Nbr','Txn_Id','Prod_Nbr']] = df_transaction_data[['Store_Nbr','Lylty_Card_Nbr','Txn_Id','Prod_Nbr']].astype(str)
# Changing the datatypes from int to str 

In [23]:
companies = ["Natural Chip Compny","Ccs",'Smiths','Kettle','Tostitos','Doritos','Pringles',"Old El Paso",
"Infzns",'Rrd','Ww','Infuzions','Red Rock','Tyrrells','Cheetos','Cobs','French','Thins','Grain','Woolworths'] # List of companies 

In [24]:
def get_match(prod_name, choices):
    match, score, ind = process.extractOne(prod_name, choices)
    return match
df_transaction_data["Brand"] = df_transaction_data["Prod_Name"].apply(lambda x : get_match(x,companies))
# Flexible Function to get the nearest match between the Prod_name Col & the companies list

In [25]:
df_transaction_data["Prod_Name"] = df_transaction_data.apply(lambda row : row['Prod_Name'].replace(row['Brand'],"")
                                                             if (row['Brand'] in row['Prod_Name']) &( pd.notnull(row['Brand'])) else row['Prod_Name'] ,axis=1 )
# Removing the co name from Prod_name Col

In [26]:
flavors = ['Chicken','Tomato Salsa','Mozzarella Basil & Pesto','Sweetchili','Original','Camembert & Fig','Supreme','Oniondip','Salt & Truffle','Spcy Bbq','Salt & Truffle','Jlpno',
'Seasalt','Siracha Lime	','Vinegar','Burger Rings','Sour Cream','Barbeque','Friedchicken','Ricotta'] # List of flavours

In [27]:
df_transaction_data['Flavor'] = df_transaction_data['Prod_Name'].apply(lambda x : get_match(x,flavors)) # creating new col with only the flavours

In [28]:
df_transaction_data.drop(columns=["Prod_Name"], inplace=True) # removing the Prod_name Col

In [29]:
df_transaction_data.Flavor = df_transaction_data.Flavor.replace('Siracha Lime\t' , 'Siracha Lime') 

In [30]:
df_purchase_behaviour.Lylty_Card_Nbr = df_purchase_behaviour.Lylty_Card_Nbr.astype(str) # changing the loyality col to STR to make the merge

In [31]:
df_all = df_transaction_data.merge(df_purchase_behaviour , on= 'Lylty_Card_Nbr' , how='outer') # merging 2 DF into one 

In [32]:
df_all[(df_all['Age_Stage'] =='New') & (df_all['Package_Size'] < 150 )].groupby(['Premium_Customer'])['Tot_Sales'].sum()

Premium_Customer
Budget        4446.3
Mainstream    3228.1
Premium       2362.6
Name: Tot_Sales, dtype: float64

In [33]:
df_all.isnull().sum() # we only have null values in Package_size but won't affect the whole column 

Date                   0
Store_Nbr              0
Lylty_Card_Nbr         0
Txn_Id                 0
Prod_Nbr               0
Prod_Qty               0
Tot_Sales              0
Package_Size        3257
Brand                  0
Flavor                 0
Premium_Customer       0
Age_Stage              0
Marital                0
Fam                    0
dtype: int64

In [129]:
df_all.to_csv("Full_data_cleaned.csv" , index= False)

In [134]:
df_all.dtypes

Date                datetime64[ns]
Store_Nbr                   object
Lylty_Card_Nbr              object
Txn_Id                      object
Prod_Nbr                    object
Prod_Qty                     int64
Tot_Sales                  float64
Package_Size               float64
Brand                       object
Flavor                      object
Premium_Customer            object
Age_Stage                   object
Marital                     object
Fam                         object
dtype: object